### **Vision computacional moderna y modelos multimodales**

Este cuaderno está pensado para **explicar conceptos** de forma **simple** y **sin GPU** usando 3 figuras:

- `car` (auto deportivo al atardecer)
- `cat` (gato pixelado)
- `puppy` (cachorro en la nieve)


> **Nota:** Este cuaderno no entrena modelos grandes ni usa GPU. Usa demostraciones pequeñas y visuales para explicar las ideas de la clase.

> **Modo recomendado (clase):** ejecutar la ruta **CPU**. Las demostraciones con modelos grandes quedan como **anexo opcional**.

#### **0. Preparación: cargar imágenes desde la carpeta `figuras`**

Este bloque busca las imágenes primero en una carpeta llamada `figuras/`y  si no existe, usa la carpeta actual.

Nombres esperados:
- `car.png`
- `cat.png`
- `puppy.png`


#### **Cómo usar este cuaderno en clase**

- Ejecuta las celdas **en orden**.
- Lee los bloques de explicación y luego completa las celdas con `TODO`.
- Todo está diseñado para correr en **CPU** (sin GPU).
- Las imágenes se buscan en:
  1. `figuras/`
  2. carpeta actual
  3. `/mnt/data`

> Consejo: si trabajas en Google Colab, sube una carpeta `figuras` con `car.png`, `cat.png` y `puppy.png`.


Al final del cuaderno deberías poder explicar, con tus palabras:

1. Por qué se pasó de CNN a **Vision Transformers (ViT)** en muchos escenarios.
2. Cómo **CLIP** aprende un espacio conjunto texto-imagen.
3. Cómo funciona (conceptualmente) **VQA**.
4. Cómo un **MLLM** conecta visión (y otras modalidades) con un LLM.


In [ ]:
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import textwrap

# Rutas candidatas (primero la carpeta 'figuras')
candidate_dirs = [
    Path("figuras"),
    Path("."),          # útil si ejecutas el notebook junto a las imágenes
    Path("/mnt/data"),  # útil en este entorno
]

expected = {
    "car": "car.png",
    "cat": "cat.png",
    "puppy": "puppy.png",
}

def find_image(name):
    filename = expected[name]
    for d in candidate_dirs:
        p = d / filename
        if p.exists():
            return p
    return None

image_paths = {name: find_image(name) for name in expected}

missing = [k for k, v in image_paths.items() if v is None]
if missing:
    raise FileNotFoundError(
        f"No encontré estas imágenes: {missing}. "
        "Colócalas en una carpeta 'figuras/' o junto al notebook."
    )

image_paths


In [ ]:
# Carga las imágenes
images = {name: Image.open(path).convert("RGB") for name, path in image_paths.items()}

# Muestra las tres imágenes
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for ax, (name, img) in zip(axes, images.items()):
    ax.imshow(img)
    ax.set_title(name)
    ax.axis("off")
plt.tight_layout()
plt.show()


#### **1. De convoluciones a Vision Transformers (ViT)**

Las **CNNs** procesan imágenes con filtros locales (convoluciones). Son muy buenas detectando bordes, texturas y patrones espaciales.

Los **Vision Transformers (ViT)** toman una idea del NLP:
- partir la entrada en **tokens**
- pasar esos tokens por un **Transformer encoder**

En ViT, los "tokens" son **parches de imagen** (patches).

##### **Idea central**
- **CNN:** aprende patrones locales con kernels.
- **ViT:** divide la imagen en parches y usa **self-attention** para modelar relaciones globales.


##### **1.1 Demostración visual: convertir una imagen en patches (como ViT)**

Usaremos la imagen del gato para ver cómo una imagen se divide en parches y luego se aplana.


In [ ]:
def patchify(img, patch_size=64):
    arr = np.array(img)
    h, w, c = arr.shape
    h2 = (h // patch_size) * patch_size
    w2 = (w // patch_size) * patch_size
    arr = arr[:h2, :w2]

    patches = []
    coords = []
    for y in range(0, h2, patch_size):
        for x in range(0, w2, patch_size):
            patches.append(arr[y:y+patch_size, x:x+patch_size])
            coords.append((x, y))
    return arr, patches, coords

cat_img = images["cat"]
cat_cropped, cat_patches, cat_coords = patchify(cat_img, patch_size=64)

print("Tamaño original:", np.array(cat_img).shape)
print("Tamaño recortado para patching:", cat_cropped.shape)
print("Número de parches:", len(cat_patches))


In [ ]:
# Muestra imagen recortada + grilla de patches
arr = cat_cropped.copy()
h, w, _ = arr.shape
patch_size = 64

fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(arr)
for x in range(0, w+1, patch_size):
    ax.axvline(x-0.5, linewidth=1)
for y in range(0, h+1, patch_size):
    ax.axhline(y-0.5, linewidth=1)
ax.set_title("Imagen del gato dividida en patches (ViT)")
ax.axis("off")
plt.show()


In [ ]:
# Muestra algunos patches en secuencia (tokens visuales)
n_show = min(8, len(cat_patches))
fig, axes = plt.subplots(1, n_show, figsize=(2*n_show, 2))
if n_show == 1:
    axes = [axes]

for i in range(n_show):
    axes[i].imshow(cat_patches[i])
    axes[i].set_title(f"P{i}")
    axes[i].axis("off")

plt.suptitle("Parches (tokens) de la imagen")
plt.tight_layout()
plt.show()


##### **1.2 Actividad guiada (ViT)-completa**

Responde brevemente:

1. ¿Qué representa un *patch* en ViT?
2. ¿Por qué se dice que los patches se vuelven "tokens visuales"?
3. ¿Qué problema aparece si aumentamos mucho la resolución de la imagen?


In [ ]:
# TODO (estudiante):
# Cambia el tamaño de patch a 32, 48 y 96.
# Observa cuántos patches aparecen y cómo cambia el detalle.

tamaños = [32, 48, 96]

for p in tamaños:
    print(f"Probando patch_size={p}")
    # Reutiliza la imagen 'cat' si existe en el entorno del cuaderno original
    # Sugerencia:
    # patches, grid = patchify(images["cat"], patch_size=p)
    # print("Número de patches:", len(patches))
    # display(grid)
    print("TODO")


In [ ]:
# TODO (estudiante):
# Completa una función que estime cuántos tokens visuales genera ViT
# para una imagen HxW con patch_size = p.

def num_patches(h, w, p):
    # return ...
    pass

ejemplos = [
    (224, 224, 16),
    (224, 224, 32),
    (384, 384, 16),
]

for h, w, p in ejemplos:
    print((h, w, p), "->", num_patches(h, w, p))


##### **1.3 ¿Qué hace ViT después de los patches?**

A nivel conceptual (sin entrenar nada aquí):

1. **Patchify** la imagen.
2. Convertir cada patch en un vector (**linear projection**).
3. Agregar información posicional (**positional embeddings**).
4. Pasar la secuencia por un **Transformer Encoder**.
5. Usar el token especial `[CLS]` (o pooling) para clasificar.



In [ ]:
## Tus respuestas


#### **2. Representaciones conjuntas texto-imagen (CLIP) y recuperación**

##### **2.1 Idea de CLIP (elemental)**

CLIP aprende dos encoders:
- un **text encoder**
- un **image encoder**

Ambos producen embeddings en el **mismo espacio vectorial**, de modo que:
- una imagen de un cachorro queda cerca de textos sobre un cachorro
- una imagen de un auto queda cerca de textos sobre autos

##### **2.2 Demostración sin un modelo grande**
Aquí haremos una **versión explicativa**:
- Definimos captions de texto.
- Calculamos "embeddings" simples para mostrar la lógica de similitud y recuperación.


In [ ]:
# Captions (puedes cambiarlos)
captions = {
    "car": "A sports car driving on the road at sunset",
    "cat": "A pixelated image of a cute cat",
    "puppy": "A puppy playing in the snow",
}

captions


In [ ]:
# Embeddings "simples" no reales
# Definimos un vocabulario pequeño para representar conceptos.
vocab = [
    "car", "road", "sunset",
    "cat", "pixelated", "cute",
    "puppy", "dog", "snow",
]

vocab_index = {w: i for i, w in enumerate(vocab)}

def text_embedding(text):
    vec = np.zeros(len(vocab), dtype=float)
    t = text.lower()
    for w in vocab:
        if w in t:
            vec[vocab_index[w]] += 1.0
    # Normalización
    norm = np.linalg.norm(vec)
    return vec / norm if norm > 0 else vec

# Para imágenes, hacemos un embedding simbólico asociado a su contenido (didáctico)
image_semantics = {
    "car": ["car", "road", "sunset"],
    "cat": ["cat", "pixelated", "cute"],
    "puppy": ["puppy", "dog", "snow"],
}

def image_embedding(image_name):
    vec = np.zeros(len(vocab), dtype=float)
    for w in image_semantics[image_name]:
        vec[vocab_index[w]] += 1.0
    norm = np.linalg.norm(vec)
    return vec / norm if norm > 0 else vec

# Construimos embeddings
img_embs = {name: image_embedding(name) for name in images}
txt_embs = {name: text_embedding(txt) for name, txt in captions.items()}

img_embs["cat"], txt_embs["cat"]


In [ ]:
def cosine(a, b):
    denom = (np.linalg.norm(a) * np.linalg.norm(b))
    if denom == 0:
        return 0.0
    return float(np.dot(a, b) / denom)

# Matriz de similitud texto-imagen (filas: textos, columnas: imágenes)
img_names = list(images.keys())
txt_names = list(captions.keys())

sim_matrix = np.zeros((len(txt_names), len(img_names)))
for i, tname in enumerate(txt_names):
    for j, iname in enumerate(img_names):
        sim_matrix[i, j] = cosine(txt_embs[tname], img_embs[iname])

sim_matrix


In [ ]:
# Visualización de la matriz de similitud (estilo CLIP retrieval)
fig, ax = plt.subplots(figsize=(7, 4))
im = ax.imshow(sim_matrix)

ax.set_xticks(range(len(img_names)))
ax.set_xticklabels(img_names)
ax.set_yticks(range(len(txt_names)))
ax.set_yticklabels([captions[t] for t in txt_names])

# Etiquetas envueltas para que no se corten
ax.set_yticklabels([textwrap.fill(captions[t], 28) for t in txt_names])

for i in range(sim_matrix.shape[0]):
    for j in range(sim_matrix.shape[1]):
        ax.text(j, i, f"{sim_matrix[i,j]:.2f}", ha="center", va="center")

ax.set_title("Matriz de similitud texto–imagen (demo tipo CLIP)")
plt.colorbar(im, ax=ax, fraction=0.045)
plt.tight_layout()
plt.show()


In [ ]:
# Recuperación texto -> imagen (image retrieval)
def retrieve_image(query_text, top_k=3):
    q = text_embedding(query_text)
    scores = []
    for iname in img_names:
        s = cosine(q, img_embs[iname])
        scores.append((iname, s))
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_k]

queries = [
    "a cute pixelated cat",
    "a dog in the snow",
    "a fast car on the road at sunset",
]

for q in queries:
    print("\nConsulta:", q)
    for iname, score in retrieve_image(q):
        print(f"  {iname:>5s}  score={score:.2f}")

#### **2.3 Demostración opcional con CLIP real**

Esta sección agrega una **versión real** usando `transformers` y el modelo **`openai/clip-vit-base-patch32`**.

Aquí se añade:

- embeddings reales texto–imagen,
- matriz de similitud real,
- recuperación **texto->imagen** e **imagen->texto**,
- métricas **Recall@K** y **mAP**.

> **Nota:** requiere `transformers` (y normalmente internet para descargar el modelo la primera vez).


In [ ]:
# [OPCIONAL] Si estás en Colab/entorno limpio, descomenta:
# !pip install transformers torch pillow matplotlib


In [ ]:
# CLIP real (texto↔imagen) usando las imágenes y captions del cuaderno
# Reutiliza:
# - image_paths (definido en la sección 0)
# - captions (definido en la sección 2)

try:
    import torch
    import numpy as np
    import matplotlib.pyplot as plt
    from PIL import Image
    from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel
except Exception as e:
    raise ImportError(
        "Faltan dependencias para CLIP real. Instala transformers/torch/pillow/matplotlib."
    ) from e

model_id = "openai/clip-vit-base-patch32"
clip_tokenizer = CLIPTokenizerFast.from_pretrained(model_id)
clip_processor = CLIPProcessor.from_pretrained(model_id)
clip_model = CLIPModel.from_pretrained(model_id)

# Orden fijo para mapear imagen <-> caption (ground truth 1:1)
keys = ["car", "cat", "puppy"]

# Cargar imágenes reales desde rutas locales del cuaderno
clip_images = [Image.open(image_paths[k]).convert("RGB") for k in keys]
clip_captions = [captions[k] for k in keys]

# Embeddings de imágenes
img_batch = clip_processor(images=clip_images, return_tensors="pt")
with torch.no_grad():
    image_embeddings = clip_model.get_image_features(pixel_values=img_batch["pixel_values"])

# Embeddings de texto
txt_batch = clip_tokenizer(
    clip_captions,
    return_tensors="pt",
    padding=True,
    truncation=True
)
with torch.no_grad():
    text_embeddings = clip_model.get_text_features(**txt_batch)

# Normalización (para coseno via producto punto)
image_embeddings = image_embeddings / image_embeddings.norm(dim=-1, keepdim=True)
text_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)

# Matriz de similitud (filas=imágenes, columnas=textos)
sim_matrix_real = (image_embeddings @ text_embeddings.T).cpu().numpy()

print("Claves:", keys)
print("Matriz de similitud real CLIP (imagen x texto):")
np.set_printoptions(precision=3, suppress=True)
print(sim_matrix_real)


In [ ]:
# Visualiza la matriz de similitud + retrieval + métricas (Recall@K, mAP)

import numpy as np
import matplotlib.pyplot as plt

# Etiquetas legibles
row_labels = [f"img:{k}" for k in keys]
col_labels = [f"txt:{k}" for k in keys]

plt.figure(figsize=(7, 5))
plt.imshow(sim_matrix_real)
plt.xticks(range(len(col_labels)), col_labels, rotation=20)
plt.yticks(range(len(row_labels)), row_labels)

for i in range(sim_matrix_real.shape[0]):
    for j in range(sim_matrix_real.shape[1]):
        plt.text(j, i, f"{sim_matrix_real[i, j]:.2f}", ha="center", va="center")

plt.title("CLIP real: matriz de similitud")
plt.tight_layout()
plt.show()


def ranks_text_to_image(sim_matrix):
    # sim_matrix: imagen x texto
    # para cada texto (columna), rankear imágenes (filas)
    ranks = []
    for j in range(sim_matrix.shape[1]):
        order = np.argsort(-sim_matrix[:, j])  # mayor similitud primero
        gt = j  # ground truth 1:1
        rank_pos = int(np.where(order == gt)[0][0]) + 1  # rank inicia en 1
        ranks.append(rank_pos)
    return ranks


def ranks_image_to_text(sim_matrix):
    # para cada imagen (fila), rankear textos (columnas)
    ranks = []
    for i in range(sim_matrix.shape[0]):
        order = np.argsort(-sim_matrix[i, :])
        gt = i
        rank_pos = int(np.where(order == gt)[0][0]) + 1
        ranks.append(rank_pos)
    return ranks


def recall_at_k(ranks, k):
    return float(np.mean([r <= k for r in ranks]))


def mean_ap_single_relevant(ranks):
    # Con 1 relevante por consulta: AP = 1/rank
    ap = [1.0 / r for r in ranks]
    return float(np.mean(ap))


r_t2i = ranks_text_to_image(sim_matrix_real)
r_i2t = ranks_image_to_text(sim_matrix_real)

print("Ranks texto->imagen:", r_t2i)
print("Ranks imagen->texto:", r_i2t)

for k in [1, 2, 3]:
    print(f"Recall@{k} texto->imagen: {recall_at_k(r_t2i, k):.3f}")
    print(f"Recall@{k} imagen->texto: {recall_at_k(r_i2t, k):.3f}")

print(f"mAP (texto->imagen, 1 relevante/consulta): {mean_ap_single_relevant(r_t2i):.3f}")
print(f"mAP (imagen->texto, 1 relevante/consulta): {mean_ap_single_relevant(r_i2t):.3f}")


# Muestra el top-k de retrieval en ambos sentidos
def topk_text_to_image(text_idx, k=3):
    order = np.argsort(-sim_matrix_real[:, text_idx])[:k]
    return [keys[i] for i in order]

def topk_image_to_text(image_idx, k=3):
    order = np.argsort(-sim_matrix_real[image_idx, :])[:k]
    return [keys[j] for j in order]

print("\nEjemplos de retrieval:")
for j, key in enumerate(keys):
    print(f"Texto '{key}' -> Top-3 imágenes:", topk_text_to_image(j, k=3))

for i, key in enumerate(keys):
    print(f"Imagen '{key}' -> Top-3 textos:", topk_image_to_text(i, k=3))


##### **2.4 Actividad guiada (CLIP real)- completa aquí**

Estas celdas permiten **replicar** la lógica de retrieval real usando `sim_matrix_real` y `keys`.

> Si no ejecutaste la demostración real, puedes completar el código de forma conceptual y comentar qué faltaría para correrlo.

In [ ]:
# TODO (estudiante): completar retrieval texto -> imagen

import numpy as np

def topk_text_to_image_estudiante(sim_matrix, keys, text_idx, k=3):
    # Pista:
    # order = np.argsort(-sim_matrix[:, text_idx])[:k]
    # return [keys[i] for i in order]
    return []

# Prueba sugerida (descomenta cuando completes):
# for j, key in enumerate(keys):
#     print(f"Texto {key} ->", topk_text_to_image_estudiante(sim_matrix_real, keys, j, k=3))

In [ ]:
# TODO (estudiante): completar retrieval imagen -> texto

def topk_image_to_text_estudiante(sim_matrix, keys, image_idx, k=3):
    # Pista:
    # order = np.argsort(-sim_matrix[image_idx, :])[:k]
    # return [keys[j] for j in order]
    return []

# Prueba sugerida:
# for i, key in enumerate(keys):
#     print(f"Imagen {key} ->", topk_image_to_text_estudiante(sim_matrix_real, keys, i, k=3))

In [ ]:
# TODO (estudiante): completar métricas de retrieval (Recall@K, mAP simplificado)

def recall_at_k_estudiante(ranks, k):
    # return float(np.mean([r <= k for r in ranks]))
    return None

def mean_ap_single_relevant_estudiante(ranks):
    # AP = 1/rank cuando hay 1 relevante por consulta
    # return float(np.mean([1.0/r for r in ranks]))
    return None

# Reto:
# 1) Construye una lista `ranks` (posición del relevante correcto en el ranking)
# 2) Reporta Recall@1, Recall@3, mAP
# 3) Interpreta el resultado

> Nota: En CLIP real, los embeddings se aprenden con **contrastive learning** usando millones de pares imagen-texto.


##### **2.5 Actividad guiada (CLIP)-interpretación de retrieval**

Observa la matriz de similitud y responde:

1. ¿Qué significa una puntuación alta entre una frase y una imagen?
2. ¿Por qué en CLIP suele ser útil normalizar embeddings?
3. ¿Qué pasa si dos captions son muy parecidos entre sí?


In [ ]:
# TODO (estudiante):
# Cambia las captions para introducir ambigüedad.
# Ejemplo: pon dos captions muy similares para 'cat' y 'puppy'
# y vuelve a ejecutar la matriz de similitud.

captions_estudiante = {
    "cat": "TODO: escribe un caption",
    "puppy": "TODO: escribe un caption",
    "car": "TODO: escribe un caption",
}

print(captions_estudiante)


In [ ]:
# TODO (estudiante):
# Implementa una recuperación simple texto -> top-k imágenes usando cosine().

def recuperar_topk(text_vec, image_vectors, k=2):
    # image_vectors: dict nombre -> vector
    # return lista ordenada [(nombre, score), ...]
    pass

# Ejemplo de uso (ajústalo con tus variables del cuaderno):
# top = recuperar_topk(text_embeds["cat"], image_embeds, k=2)
# print(top)


##### **Ejercicios textuales (sin código)**

- Explica la diferencia entre:
  - **Clasificación supervisada** (cat/dog)
  - **Recuperación multimodal** (texto <-> imagen)
- Da un ejemplo real de uso de CLIP en:
  - e-commerce.
  - moderación de contenido.
  - búsqueda en repositorios multimedia.


#### **3. Modelos visión-lenguaje y VQA (Visual Question Answering)**

Un sistema VQA recibe:
- una **imagen**
- una **pregunta en texto**

y produce una **respuesta en texto**.

Ejemplo:

- Imagen: auto
- Pregunta: *What do you see?*
- Respuesta: *A sports car driving on the road at sunset.*

##### **3.1 Demostración elemental (reglas + conocimiento visual mínimo)**
Esta demostración **no es un modelo profundo**, pero sirve para explicar el flujo completo.


In [ ]:
# Base de conocimiento simple (hecha a mano) para explicar VQA
visual_facts = {
    "car": {
        "objects": ["car", "road", "sunset"],
        "scene": "A sports car driving on the road at sunset.",
        "weather": "clear sky with sunset light",
        "dominant_topic": "transportation"
    },
    "cat": {
        "objects": ["cat", "pixel-art", "cute"],
        "scene": "A pixelated image of a cute cat.",
        "weather": "not applicable",
        "dominant_topic": "animal"
    },
    "puppy": {
        "objects": ["puppy", "snow", "park"],
        "scene": "A puppy sitting in the snow.",
        "weather": "snowy",
        "dominant_topic": "animal"
    },
}

def vqa_demo(image_name, question):
    q = question.lower().strip()
    facts = visual_facts[image_name]

    if "what" in q and ("see" in q or "image" in q or "picture" in q):
        return facts["scene"]
    if "animal" in q:
        return "Yes." if facts["dominant_topic"] == "animal" else "No."
    if "weather" in q or "snow" in q:
        return f"Weather/scene condition: {facts['weather']}."
    if "objects" in q or "things" in q:
        return "I can identify: " + ", ".join(facts["objects"]) + "."
    if "car" in q:
        return "Yes, there is a car." if "car" in facts["objects"] else "No, I do not see a car."
    return "I need a more specific question (what, objects, weather, animal, car)."


In [ ]:
# Prueba VQA en las tres imágenes
sample_questions = [
    "What do you see in this picture?",
    "What objects are visible?",
    "Is there an animal?",
    "What is the weather?",
    "Is there a car?",
]

for name in ["car", "cat", "puppy"]:
    print("\n" + "="*60)
    print("Imagen:", name)
    for q in sample_questions:
        print("Q:", q)
        print("A:", vqa_demo(name, q))


##### **3.2 ¿Cómo sería VQA moderno (real)?**
Un modelo moderno hace algo parecido a esto:

1. **Encoder visual** (por ejemplo ViT) produce representaciones de la imagen.
2. **Encoder/LLM de texto** procesa la pregunta.
3. Un módulo de fusión (cross-attention, Q-Former, etc.) combina ambos.
4. El decodificador/LLM genera la respuesta.



##### **3.3 Actividad guiada (VQA)-diseño de preguntas**

Formula 2 preguntas por imagen:

- una **descriptiva** (qué hay en la imagen),
- una **inferencial** (qué podría estar pasando).

Luego compáralas con la respuesta de la demostración y comenta:

- ¿Qué puede responder bien?
- ¿Qué no puede responder bien una demo basada en reglas?


In [ ]:
# TODO (estudiante):
# Crea una función para evaluar preguntas "fuera de dominio"
# (por ejemplo, pedir color exacto, velocidad exacta, marca, etc.)

preguntas_prueba = [
    ("car", "¿Qué objeto aparece?"),
    ("car", "¿Qué velocidad exacta lleva?"),
    ("cat", "¿Es un gato pixelado?"),
    ("puppy", "¿Qué raza exacta es?"),
]

# Sugerencia: recorre preguntas_prueba y llama a la función VQA del cuaderno
for item in preguntas_prueba:
    print(item, "-> TODO")


##### **Ejercicio adicional**

Describe, en 5-7 líneas, qué componentes adicionales necesitaría un sistema VQA moderno:

- encoder visual,
- encoder de texto / tokenizer,
- módulo de fusión,
- decoder o clasificador,
- datos anotados.


#### **4. LLM multimodales (MLLM): texto, imagen, audio, vídeo**

Un **MLLM** extiende un LLM para aceptar varias modalidades:

- **Texto** (prompt)
- **Imagen**
- **Audio**
- **Vídeo**

El problema central es: **¿cómo convertir cada modalidad a un formato que el LLM pueda usar?**

##### **4.1 Solución típica (alto nivel)**
- Imagen -> **Vision encoder** (ViT)
- Audio -> **Audio encoder** (por ejemplo, espectrogramas + transformer)
- Vídeo -> encoder visual-temporal (frames + tiempo)
- Luego un **módulo puente** (por ejemplo **Q-Former**) adapta esas representaciones para el LLM.


##### **4.2 Q-Former (intuición elemental)**

El **Q-Former** actúa como un **adaptador multimodal** y se sigue este proceso:

- Un **Vision Transformer preentrenado** extrae características visuales.
- Un **Q-Former entrenable** aprende consultas útiles ("queries") sobre esas características.
- Esas salidas se **proyectan** al espacio del **LLM preentrenado**.
- El LLM genera texto (descripción, respuesta VQA, resumen, etc.).


In [ ]:
# Simulación conceptual de un pipeline MLLM (texto, imagen, audio, vídeo)
# No ejecuta modelos grandes: solo muestra el flujo.

def mllm_router(modality, payload):
    if modality == "text":
        return {"encoder": "Text encoder/tokenizer", "output": "text embeddings"}
    elif modality == "image":
        return {"encoder": "Vision encoder (ViT)", "output": "visual embeddings"}
    elif modality == "audio":
        return {"encoder": "Audio encoder", "output": "audio embeddings"}
    elif modality == "video":
        return {"encoder": "Video encoder (frames + temporal modeling)", "output": "video embeddings"}
    else:
        return {"encoder": "Unknown", "output": "N/A"}

modalities = ["text", "image", "audio", "video"]
for m in modalities:
    step = mllm_router(m, None)
    print(f"{m:>5s} -> {step['encoder']} -> {step['output']} -> adapter/Q-Former -> LLM -> texto")


##### **4.3 Ruta **CPU-only** (sin BLIP-2 pesado)**

Esta ruta refuerza el patrón multimodal (texto + imagen + audio + video) con **datos ligeros y reglas explicables**.  

Sirve como práctica principal del curso cuando no se dispone de GPU.

> **Sugerencia:** usa esta sección solo como demostración final o actividad opcional en Colab con GPU.

In [ ]:
# Mini-lab CPU-only: respuesta multimodal con metadatos (didáctico)

audio_db = {
    "audio_1": {"evento": "ladrido", "duracion_s": 2.3, "confianza": 0.88},
    "audio_2": {"evento": "motor", "duracion_s": 1.1, "confianza": 0.73},
}

video_db = {
    "video_1": {"evento": "auto_en_movimiento", "frames_clave": ["car", "sunset"]},
    "video_2": {"evento": "gato_sentado", "frames_clave": ["cat", "pixel-art"]},
}

def responder_multimodal_cpu(prompt, imagen_id=None, audio_id=None, video_id=None):
    prompt_l = prompt.lower()
    partes = []

    if imagen_id is not None and "captions" in globals() and imagen_id in captions:
        partes.append(f"Imagen: {captions[imagen_id]}")

    if audio_id is not None and audio_id in audio_db:
        a = audio_db[audio_id]
        partes.append(f"Audio: '{a['evento']}' ({a['duracion_s']} s, conf={a['confianza']:.2f})")

    if video_id is not None and video_id in video_db:
        v = video_db[video_id]
        partes.append(f"Video: '{v['evento']}' con pistas {v['frames_clave']}")

    if "duración" in prompt_l or "duracion" in prompt_l:
        if audio_id in audio_db:
            return f"La duración del audio es {audio_db[audio_id]['duracion_s']} s."
        return "No tengo audio seleccionado."

    if "evento" in prompt_l:
        if video_id in video_db:
            return f"El evento principal del video es: {video_db[video_id]['evento']}."
        return "No tengo video seleccionado."

    if "qué animal" in prompt_l or "que animal" in prompt_l:
        if imagen_id == "puppy":
            return "Parece un cachorro."
        if imagen_id == "cat":
            return "Parece un gato."
        return "No identifico un animal claro."

    return " | ".join(partes) if partes else "Sin modalidades activas."

print(responder_multimodal_cpu("Describe la escena", imagen_id="car", video_id="video_1"))
print(responder_multimodal_cpu("¿Qué animal aparece?", imagen_id="puppy", audio_id="audio_1"))
print(responder_multimodal_cpu("¿Cuál es la duración del audio?", audio_id="audio_1"))

In [ ]:
# TODO (estudiante): completa aquí
# 1) Agrega un nuevo audio y un nuevo video.
# 2) Extiende `responder_multimodal_cpu` con una nueva intención (por ejemplo: "confianza" o "frames").
# 3) Prueba con un prompt propio.

# Tu prueba:
# print(...)

##### **4.4 Anexo opcional (GPU) con **BLIP-2** (captioning + VQA) y puente a Q-Former**

Esta sección agrega código real para conectar lo conceptual con un modelo visión–lenguaje moderno.

Esta sección:

- refuerza el patrón **encoder visual + adaptador/Q-Former + LLM**,
- muestra **captioning** (generación anclada en imagen),
- muestra **VQA** (pregunta-respuesta sobre imagen).

##### **Recordatorio conceptual (BLIP-2 en dos etapas)**

1. **Etapa 1:** aprender consultas (*queries*) que extraen información útil del encoder visual.  
2. **Etapa 2:** proyectar esas consultas al espacio del LLM para generar texto condicionado.

> **Importante:** `Salesforce/blip2-opt-2.7b` suele requerir **GPU** y bastante memoria. Deja esta sección como **opcional** en clase si el entorno es CPU.

> **Sugerencia:** usa esta sección solo como demo final o actividad opcional en Colab con GPU.

In [ ]:
# [OPCIONAL] Si estás en Colab / entorno limpio, descomenta:
# !pip install transformers accelerate sentencepiece


In [ ]:
# Carga BLIP-2 (opcional, normalmente con GPU)

#try:
#    import torch
#    from transformers import AutoProcessor, Blip2ForConditionalGeneration
#except Exception as e:
#    raise ImportError("Faltan dependencias para BLIP-2. Instala transformers/torch.") from e

#BLIP_MODEL_ID = "Salesforce/blip2-opt-2.7b"
#BLIP_REV = "51572668da0eb669e01a189dc22abe6088589a24"  # revisión estable usada en el cuaderno fuente

#device = "cuda" if torch.cuda.is_available() else "cpu"
#dtype = torch.float16 if device == "cuda" else torch.float32

#print("Dispositivo:", device)
#print("dtype:", dtype)

#blip_processor = AutoProcessor.from_pretrained(BLIP_MODEL_ID, revision=BLIP_REV)

# Nota: en CPU puede tardar mucho o no entrar en memoria. Mantener opcional.
#blip_model = Blip2ForConditionalGeneration.from_pretrained(
#    BLIP_MODEL_ID,
#    revision=BLIP_REV,
#    torch_dtype=dtype
#)
#blip_model.to(device)
#print("BLIP-2 cargado.")


In [ ]:
# Demostración 1: Captioning (imagen -> texto)
#from PIL import Image

# Reutilizamos la imagen local del auto
#img_car = Image.open(image_paths["car"]).convert("RGB")

#inputs = blip_processor(img_car, return_tensors="pt")
#if "pixel_values" in inputs:
#    inputs["pixel_values"] = inputs["pixel_values"].to(device, dtype)
#if "input_ids" in inputs:
#    inputs["input_ids"] = inputs["input_ids"].to(device)
#if "attention_mask" in inputs:
#    inputs["attention_mask"] = inputs["attention_mask"].to(device)

#with torch.no_grad():
#    gen_ids = blip_model.generate(**inputs, max_new_tokens=30)

#caption_real = blip_processor.batch_decode(gen_ids, skip_special_tokens=True)[0].strip()
#print("Caption BLIP-2:", caption_real)


# Demostración 2: VQA (imagen + pregunta -> respuesta)
#prompt = "Question: What do you see in this image? Answer:"
#inputs_vqa = blip_processor(img_car, text=prompt, return_tensors="pt")

#if "pixel_values" in inputs_vqa:
#    inputs_vqa["pixel_values"] = inputs_vqa["pixel_values"].to(device, dtype)
#if "input_ids" in inputs_vqa:
#    inputs_vqa["input_ids"] = inputs_vqa["input_ids"].to(device)
#if "attention_mask" in inputs_vqa:
#    inputs_vqa["attention_mask"] = inputs_vqa["attention_mask"].to(device)

#with torch.no_grad():
#    vqa_ids = blip_model.generate(**inputs_vqa, max_new_tokens=40)

#answer_real = blip_processor.batch_decode(vqa_ids, skip_special_tokens=True)[0].strip()
#print("Respuesta VQA BLIP-2:", answer_real)


In [ ]:
# (Opcional) Prompt encadenado tipo chat
# Útil para mostrar memoria textual simple sobre la misma imagen.

#prompt_chat = (
#    "Question: What do you see in this image? "
#    "Answer: A sports car driving on the road at sunset. "
#    "Question: What colors are prominent? "
#    "Answer:"
#)

#inputs_chat = blip_processor(img_car, text=prompt_chat, return_tensors="pt")
#if "pixel_values" in inputs_chat:
#    inputs_chat["pixel_values"] = inputs_chat["pixel_values"].to(device, dtype)
#if "input_ids" in inputs_chat:
#    inputs_chat["input_ids"] = inputs_chat["input_ids"].to(device)
#if "attention_mask" in inputs_chat:
#    inputs_chat["attention_mask"] = inputs_chat["attention_mask"].to(device)

#with torch.no_grad():
#    chat_ids = blip_model.generate(**inputs_chat, max_new_tokens=40)

#chat_answer = blip_processor.batch_decode(chat_ids, skip_special_tokens=True)[0].strip()
#print("Salida (prompt encadenado):", chat_answer)


##### **4.5 Actividad guiada (MLLM)-extender a audio y vídeo (opcional)**

Completa la tabla conceptual:

| Modalidad | Ejemplo de entrada | Encoder típico | Salida posible |
|---|---|---|---|
| Texto | ... | ... | ... |
| Imagen | ... | ... | ... |
| Audio | ... | ... | ... |
| Vídeo | ... | ... | ... |

Puedes responder en una celda Markdown nueva.


In [ ]:
# TODO (estudiante):
# Simula un "router multimodal" que detecte la modalidad
# y devuelva qué encoder usarías (solo texto, sin ML real).

def elegir_encoder(modalidad):
    modalidad = modalidad.lower().strip()
    # Completa:
    # if modalidad == "texto": return ...
    # ...
    return "TODO"

for m in ["texto", "imagen", "audio", "video"]:
    print(m, "->", elegir_encoder(m))


In [ ]:
# TODO (estudiante, opcional):
# Implementa una versión más completa del pipeline conceptual:
# entrada -> encoder por modalidad -> proyección -> "LLM"
# Usa strings y diccionarios, no hace falta deep learning real.

class MiniMLLM:
    def __init__(self):
        self.encoders = {
            "texto": "TextEncoder",
            "imagen": "VisionEncoder",
            "audio": "AudioEncoder",
            "video": "VideoEncoder",
        }

    def procesar(self, modalidad, dato):
        # TODO
        return {
            "modalidad": modalidad,
            "encoder": "TODO",
            "embedding": "TODO",
            "respuesta": "TODO"
        }

demo = MiniMLLM()
print(demo.procesar("imagen", "car.png"))


#### **5. Ejercicios de evaluación rápida**

##### **Conceptuales**

1. ¿Por qué ViT usa patches en lugar de convoluciones?
2. ¿Qué optimiza CLIP durante el entrenamiento?
3. ¿Qué diferencia hay entre VQA y captioning?
4. ¿Qué papel cumple un módulo tipo **Q-Former**?

##### **De aplicación**

1. Propón un caso de uso educativo con imagen + texto.
2. Propón un caso de uso industrial con imagen + audio.
3. Explica cuándo **no** usarías un MLLM (coste, datos, latencia, privacidad).


In [ ]:
## Tus respuestas